Name - Rutik Kothwala
UID - U01868702
Subject - CS668, Analytics Capstone

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import string
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
drugs_reviews= pd.read_csv("/content/drugs_reviews.csv") # loading the preprocessed reviews saved dataset

In [ ]:
# Remove rows with NaN values in the 'preprocessed_review' column
drugs_reviews.dropna(subset=['preprocessed_review'], inplace=True)

# Reset the index if needed
drugs_reviews.reset_index(drop=True, inplace=True)

In [ ]:
# Convert rating column to positive (1) or negative (0)
drugs_reviews['sentiment'] = drugs_reviews['rating'].apply(lambda x: 1 if x >= 6 else 0)

In [ ]:
X = drugs_reviews.drop(['review','rating','date','sentiment'],axis=1)
y = drugs_reviews['sentiment'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Modelling

# Deep Learning


In [ ]:
words_to_index = tokenizer.word_index

In [ ]:
max_sequence_length = 200

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['preprocessed_review'])

X_train_sequences = tokenizer.texts_to_sequences(X_train['preprocessed_review'])
X_test_sequences = tokenizer.texts_to_sequences(X_test['preprocessed_review'])

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# Define hyperparameters
embedding_dim = 100
lstm_units = 128
dropout_rate = 0.5

# Build the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(units=lstm_units))
model.add(Dense(64, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))

# Compile the model with Adam optimizer and a lower learning rate
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 200, 100)          6309900   
                                                                 
 lstm_6 (LSTM)               (None, 128)               117248    
                                                                 
 dense_12 (Dense)            (None, 64)                8256      
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 6435469 (24.55 MB)
Trainable params: 6435469 (24.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
batch_size = 70
epochs=50

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/Train/model1'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model=model.fit(X_train_padded, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_padded, y_test),callbacks=[model_checkpoint])

Epoch 1/50
2091/2091 [==============================] - 57s 26ms/step - loss: 0.6149 - accuracy: 0.6999 - val_loss: 0.6151 - val_accuracy: 0.6956
Epoch 2/50
2091/2091 [==============================] - 26s 13ms/step - loss: 0.6125 - accuracy: 0.7002 - val_loss: 0.6146 - val_accuracy: 0.6956
Epoch 3/50
2091/2091 [==============================] - 30s 14ms/step - loss: 0.6114 - accuracy: 0.7003 - val_loss: 0.6145 - val_accuracy: 0.6957
Epoch 4/50
2091/2091 [==============================] - 28s 13ms/step - loss: 0.6110 - accuracy: 0.7003 - val_loss: 0.6144 - val_accuracy: 0.6957
Epoch 5/50
2091/2091 [==============================] - 25s 12ms/step - loss: 0.6107 - accuracy: 0.7003 - val_loss: 0.6144 - val_accuracy: 0.6957
Epoch 6/50
2091/2091 [==============================] - 24s 11ms/step - loss: 0.6107 - accuracy: 0.7003 - val_loss: 0.6145 - val_accuracy: 0.6957
Epoch 7/50
2091/2091 [==============================] - 24s 12ms/step - loss: 0.6106 - accuracy: 0.7003 - val_loss: 0.6145 -

In [ ]:
best_model = load_model('/content/drive/MyDrive/Train/model1')
y_pred_prob = best_model.predict(X_test_padded)
y_pred = (y_pred_prob > 0.5).astype(int)
report = classification_report(y_test, y_pred)
print(report)

1144/1144 [==============================] - 5s 4ms/step
              precision    recall  f1-score   support

           0       0.88      0.80      0.84     11138
           1       0.92      0.95      0.93     25450

    accuracy                           0.91     36588
   macro avg       0.90      0.88      0.89     36588
weighted avg       0.91      0.91      0.91     36588



In [ ]:
# Deep Learning LSTM model gives the accuracy of 0.91 with just 50 epochs

In [ ]:
df_model4= pd.read_csv("/content/df_model4.csv")

In [ ]:
df_model4['model5'] = y_pred # adding the result of Deep learning LSTM model to df_model4 file in which all result of machine lerning model are stored

In [ ]:
df_model4.to_csv('df_model5.csv') 

In [ ]:
df_model4.to_csv('/content/df_model5.csv') 

In [ ]:
df_model5= pd.read_csv("/content/df_model5.csv") 

# Recommendation

Loading the df_model5 file which contains results of best model from each feature extraction technique, deep learning model and other features.

In [ ]:
df_model5.head()

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,drugName,condition,usefulCount,preprocessed_review,model2,model3,model4,model5
0,0,0,0,43093,50678,Cymbalta,Depression,28,day 1 cymbalta it039 aw didn039t sleep last ni...,0,0,0,1
1,1,1,1,145643,9815,Adderall,ADHD,41,take 3 month feel get somewher take 15 mg morn...,1,1,1,0
2,2,2,2,46018,54104,Neurontin,Anxiety,119,notic take away anxieti leav littl foggi it039...,1,1,1,1
3,3,3,3,157442,23639,Fluoxetine,Panic Disorde,79,fluoxetin pretti gentl compar antidepress medi...,1,1,1,1
4,4,4,4,13234,15535,MiraLax,Constipation,104,medic heaven sent suffer bad constip famili ea...,1,1,0,0


Normalizing Useful Count

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()


df_model5['normalized_useful_count'] = scaler.fit_transform(df_model5[['usefulCount']])

print(df_model5.head())

   Unnamed: 0.4  Unnamed: 0.3  Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  \
0             0             0             0         43093       50678   
1             1             1             1        145643        9815   
2             2             2             2         46018       54104   
3             3             3             3        157442       23639   
4             4             4             4         13234       15535   

     drugName      condition  usefulCount  \
0    Cymbalta     Depression           28   
1    Adderall           ADHD           41   
2   Neurontin        Anxiety          119   
3  Fluoxetine  Panic Disorde           79   
4     MiraLax   Constipation          104   

                                 preprocessed_review  model2  model3  model4  \
0  day 1 cymbalta it039 aw didn039t sleep last ni...       0       0       0   
1  take 3 month feel get somewher take 15 mg morn...       1       1       1   
2  notic take away anxieti leav littl foggi it039.

In [ ]:
# Normalizing the usefulcount before using it for our recommendation score to get the unbias result.

In [ ]:
df_model5['recommendation_score'] =(df_model5['model2'] + df_model5['model3'] + df_model5['model4'] + df_model5['model5'] )*df_model5['normalized_useful_count']

In [ ]:
# So in above to get the recommendation score we have combine the result of  best model from each feature extraction technique also the result of deep learning model and then multiply it by normalize useful count

In [ ]:
# grouping the datafrom with respect to condition and drugname.

df_rec = df_model5.groupby(['condition','drugName']).agg({'recommendation_score':['sum']})

In [ ]:
df_rec.head()

recommendation_score
                                                           sum
condition drugName                                            
ADHD      Adderall                                    7.183230
          Adderall XR                                 4.630435
          Amphetamine                                 0.549689
          Amphetamine / dextroamphetamine            11.068323
          Armodafinil                                 0.642857

In [ ]:
df_rec.tail()

recommendation_score
                                                       sum
condition                drugName                         
von Willebrand's Disease Desmopressin             0.024845
                         Stimate                  0.024845
zen Shoulde              Diclofenac               0.015528
                         Nabumetone               0.177019
                         Naproxen                 0.018634

In [ ]:
df_rec

recommendation_score
                                                                          sum
condition                drugName                                            
ADHD                     Adderall                                    8.607143
                         Adderall XR                                 5.083851
                         Amphetamine                                 0.635093
                         Amphetamine / dextroamphetamine            12.888199
                         Armodafinil                                 0.709627
...                                                                       ...
von Willebrand's Disease Desmopressin                                0.024845
                         Stimate                                     0.024845
zen Shoulde              Diclofenac                                  0.015528
                         Nabumetone                                  0.177019
                         Naproxen                                    0.018634

[3798 rows x 1 columns]

So above is recommendation system which suggest best medicine for the given condition higher is the score better is the drug for the given condition.

Conclusion: The Random Forest model excels in drug recommendation frameworks with a remarkable performance across various feature extraction techniques with the highest 0.91 with TF-IDF.
Deep learning models are a promising alternative to basic machine learning models matching the best accuracy with just 50 epochs. 
Future works may involve employing different oversampling techniques to enhance recall value and hypertuning can be done to optimize model performance.
Furthermore, instead of combining model results, using proper ensemble techniques could give a more robust recommendation score.
